# Texas Tow Trucks

We're going to scrape some [tow trucks in Texas](https://www.tdlr.texas.gov/tools_search/). 

# Part One: Building a company list

Search for businesses with the word **WRECK** in their names.

* **Tip:** Start by scraping the first page to a dataframe, then expand to a loop that combines all of the pages. Finally combine all of the dataframes with `pd.concat`. You might find [this page helpful](https://jonathansoma.com/everything/scraping/pagination/), although the "On an interactive site" example uses Selenium instead of Playwright. You should be able to figure out how to change it!
* **Tip:** You can't just do a `try`/`except`, because even if you ask for page 99999 it will always give you the last page again! Watch out that you don't get stuck in an infinite loop!

In [48]:
import re
from bs4 import BeautifulSoup
from playwright.async_api import async_playwright

In [49]:
import pandas as pd

In [50]:
import csv

In [182]:
import asyncio

In [51]:
playwright = await async_playwright().start()
browser = await playwright.chromium.launch(headless = False)
page = await browser.new_page()

In [20]:
await page.goto("https://www.tdlr.texas.gov/tools_search/")

<Response url='https://www.tdlr.texas.gov/tools_search/' request=<Request url='https://www.tdlr.texas.gov/tools_search/' method='GET'>>

In [21]:
await page.locator("#namebutton").click()

In [22]:
await page.locator("#namedata").fill("WRECK")

In [23]:
await page.locator("#submit3").click()

In [41]:
with open('WRECK.csv', 'w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    page_num = 1

    for page_num in range(1, 42):
        html = await page.content()
        doc = BeautifulSoup(html, 'html.parser')  # Specify parser
        table = doc.select('tbody')[2]
        table_rows = table.find_all('tr')

        for row in table_rows:
            cols = [ele.text.strip() for ele in row.find_all('td')]
            writer.writerow(cols)

        if page_num < 41:
            next_button_selector = 'a[href*="mccs_search_process.asp?page="]:text("[Next >>]")'
            await page.wait_for_selector(next_button_selector, state="visible")
            await page.click(next_button_selector, timeout=60000)


### Cleanup

If you haven't already, rename the columns to be:
    
    * Customer
    * DBA Name
    * TDLR Number
    * City
    * State
    * Zip code

and remove all of the rows where the customer name is `Customer`.

In [44]:
df = pd.read_csv('WRECK.csv')

In [46]:
df = df[df['Customer'] != 'Customer']
df.to_csv('wreckers.csv', index=False)

## Save as `wreckers.csv`

# Part Two: Company info

> You can use whatever tool you'd like for this, but form submission doesn't necessarily mean Playwright! If you want to go the `requests` route instead, feel free to [look at this page](https://jonathansoma.com/everything/scraping/pretending-to-be-a-browser/) about several ways to pretend to be a browser.

## Step 1: Scraping one page

Try searching from the [tools page](https://www.tdlr.texas.gov/tools_search/) for the TDLR Number `006556161C`. From the results page, scrape the:

* Business name
* Phone number
* License status
* Physical address

And save the results into a dictionary. Also include `TDLR Number` in the dictionary. **Print the dictionary**.

> ***Tip:** It's best if each item has its own key, but **it's fine to pull "larger" sections of the page and split them up in pandas later on***
>
> ***Tip:** Be sure you don't forget to include `TDLR Number` in the dictionary!*

In [154]:
await page.goto("https://www.tdlr.texas.gov/tools_search/")

<Response url='https://www.tdlr.texas.gov/tools_search/' request=<Request url='https://www.tdlr.texas.gov/tools_search/' method='GET'>>

In [155]:
await page.locator("#mcrbutton").click()
await page.locator("#mcrdata").fill("006556161C")
await page.locator("#submit3").click()

In [156]:
html = await page.content()
soup = BeautifulSoup(html, 'html.parser')
big_table = soup.find('table')
small_table = big_table.find_all('table')[2]
cer_table = big_table.find_all('table')[3]

company_info = {
    'TDLR_Number': '006556161C',
    'name':small_table.find_all('td')[2].find('strong', string=lambda text: text and 'Name:' in text).next_sibling.strip(),
    'number':small_table.find_all('td')[8].find('strong', string='Phone:').next_sibling.strip(),
    'status':cer_table.find_all('td')[1].find_next('font').get_text(strip=True),
    'cer_num':cer_table.find_all('td')[2].find('strong', string=lambda text: 'Number:' in text).find_next('b').get_text(strip=True)
                                                        }

In [157]:
company_info

{'TDLR_Number': '006556161C',
 'name': 'DIRTY SOUTH TRANSPORT AND RECOVERY, LLC',
 'number': '713-259-5445',
 'status': 'Active',
 'cer_num': '006556161C'}

## Step 2: Move into one cell

Move the code above all into one cell that relies on the variable `tdlr_number`.

Add the code below to the page and confirm that it displays the data for the correct result.

```python
tdlr_number = '0654479VSF'
```

Confirm that the information is correct. Did it not work out? Go back and edit your selectors, or be a little broader in the parts of the page you sweep up.

In [176]:
async def get_tdlr(tdlr_number):
    async with async_playwright() as p:
        browser = await p.chromium.launch()
        page = await browser.new_page()
        await page.goto("https://www.tdlr.texas.gov/tools_search/")
        await page.locator("#mcrbutton").click()
        await page.locator("#mcrdata").fill(tdlr_number)
        await page.locator("#submit3").click()
        html = await page.content()
        soup = BeautifulSoup(html, 'html.parser')

        company_info = {
            'TDLR_Number': tdlr_number,
            'name': soup.find('strong', string=lambda text: 'Name:' in text).next_sibling.strip(),
            'number': soup.find('strong', string='Phone:').next_sibling.strip(),
            'status': soup.find('strong', string='Status:').find_next('font').get_text(strip=True),
        }
        await browser.close()
        return company_info

In [170]:
await get_tdlr('006564940C')

{'TDLR_Number': '006564940C',
 'name': 'A&NTOWINGLLC',
 'number': '2106678546',
 'status': 'Active'}

## Step 3: Scraping many pages

Using pandas, read in `trucks-subset.csv`.

In [166]:
pd.read_csv('trucks-subset.csv')

,TDLR Number
0,006565540C
1,0654479VSF
2,006564940C


## Scrape every single row, creating a list of dictionaries from the scraped data.

You should never use for loops with pandas *except when working with Playwright*. To use loops in pandas you'll make use of `.iterrows()`.

The code below loops through a dataframe called `df` and prints out the `address` column.

```python
for index, row in df.iterrows():
    print(row['address'])
```

You'll adapt this code to use your dataframe, and combine it with the scraping code you wrote above.

> ***Tip:** This is like what we did for the townships in class*

In [164]:
df = pd.read_csv('trucks-subset.csv')

In [167]:
df.head()

,TDLR Number
0,006565540C
1,0654479VSF
2,006564940C


In [186]:
tdlr_numbers = df['TDLR Number'].tolist()

In [189]:
company_list = []
for number in tdlr_numbers:
    company_list.append(await get_tdlr(number))

In [190]:
company_list

[{'TDLR_Number': '006565540C',
  'name': 'H & A TOWING LLC',
  'number': '512-999-8883',
  'status': 'Expired'},
 {'TDLR_Number': '0654479VSF',
  'name': '24/7TOWINGANDRECOVERYLLC',
  'number': '4325576733',
  'status': 'Active'},
 {'TDLR_Number': '006564940C',
  'name': 'A&NTOWINGLLC',
  'number': '2106678546',
  'status': 'Active'}]

## Save your dataframe as `data-uncleaned.csv`

In [191]:
df = pd.DataFrame(company_list)
df.to_csv('data-uncleaned.csv')

# Cleaning your data

## Re-open the `data-uncleaned.csv` file

You probably want to set `pd.options.display.max_colwidth`

## Clean it up!

Make sure there are columns for

- Business name
- Phone number
- License status
- Physical address

And drop all of the other columns (The easiest way is to use `df = df.drop(columns=[...])`)

## Combine with the original CSV file

To combine dataframes based on indexes, you use `df.join(other_df)`. If you'd prefer to match by columns, you could also use `df.merge(other_df, left_on='...', right_on='...')` and tell it the two columns that match between the two.

### Save to a CSV